In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Get RT-DETR
if not os.path.isdir("RT-DETR"):
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip
!pip install -r requirements.txt
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

In [ ]:
# Copy and unzip data from GDrive
!cp /content/drive/MyDrive/datasets/soccertrack.zip .
!unzip -q soccertrack.zip

In [ ]:
import torch
device = torch.cuda.is_available()
print(f"GPU device for training / inference: {device}")